## First approach

Imports:

In [1]:
import jinja2
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
import ssb_pubmd as ssb

1. Define article model

In [2]:
class MarkdownBlock(BaseModel):
    type: str = "markdown"
    markdown_text: str = Field(description="The markdown text of the block.")


class HighchartBlock(BaseModel):
    type: str = "highchart"
    key: str = Field(pattern=r"^hc:", description="A unique key for the highchart.")
    data: ssb.Highchart = Field(description="The metadata of the highchart.")
    dataframe_code: str = Field(
        "The code to get from the data source to the dataframe used in the highchart, given as one block with necessary imports at top and the final dataframe stored in a variable 'df'."
    )


class FactboxBlock(BaseModel):
    type: str = "factbox"
    key: str = Field(pattern=r"^fb:", description="A unique key for the highchart.")
    data: ssb.Factbox = Field(description="The metadata of the factbox.")
    markdown_text: str = Field(description="The markdown content of the factbox.")


class StatisticalArticle(BaseModel):
    title: str = Field(description="A title for the article.")
    ingress: str = Field(description="A short ingress for the article.")
    blocks: list[MarkdownBlock | HighchartBlock | FactboxBlock]

2. Create and run agent with article as output

In [ ]:
agent = Agent(
    model=OpenAIChatModel("gpt-5-mini"),
    output_type=StatisticalArticle,
)

result = await agent.run(
    "Skriv en statistisk artikkel på norsk om klimadata, med en faktaboks og ett highcharts. Jeg har en csv-fil med kolonner 'year', 'temperature' og 'co2' - bruk pandas til å hente data for highchartet."
)

article: StatisticalArticle = result.output

{'title': 'Klimadata: temperatur og CO2 over tid',
 'ingress': 'En statistisk gjennomgang av et enkelt klimadatsett som inneholder årstall, gjennomsnittstemperatur og atmosfærisk CO₂. Artikkelen viser hvordan du kan laste inn data med pandas, lage visualiseringer og tolke trender og sammenhenger.',
 'blocks': [{'type': 'markdown',
   'markdown_text': 'Innledning\n\nKlimaendringer er et komplekst system der økende konsentrasjoner av drivhusgasser som CO₂ henger sammen med endringer i global temperatur. Her analyserer vi et enkelt datasett med kolonnene \'year\', \'temperature\' og \'co2\'. Formålet er å vise hvordan man kan beskrive dataene, visualisere utviklingen over tid og vurdere sammenhenger mellom temperatur og CO₂.\n\nDatagrunnlag og metode\n\nDatasettet forventes å ha tre kolonner:\n- year: årstall (f.eks. 1980, 1990, ...)\n- temperature: gjennomsnittlig temperatur (enhet avhengig av kilden, f.eks. °C eller avvik fra en baseline)\n- co2: CO₂-konsentrasjon i atmosfæren (vanligvi

3. Insert article output into markdown template

In [12]:
environment = jinja2.Environment(loader=jinja2.FileSystemLoader(""))
template = environment.get_template("template.qmd")
output = template.render(article=article)
with open("output.qmd", "w", encoding="utf-8") as f:
    f.write(output)

Next steps:

1. How can we change the model to fit SSB articles better?
2. How can we make the user fill in what they want and the AI fill out the rest?
3. How can we make the style similar to existing articles?